d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

-sandbox
## Managing Nested Data with Spark SQL

In this notebook, you'll be digging into mock data from a group of data centers. A **data center** is a dedicated space where computing and networking equipment is set up to collect, store, process, and distribute data. The continuous operation of centers like this can be crucial to maintaining continuity in business, so environmental conditions must be closely monitored. 

This example uses mock data from 4 different data centers, each with four different kinds of sensors that periodically collect temperature and CO<sub>2</sub> level readings. Temperature and CO<sub>2</sub> levels are stored as arrays where temperature is collected 12 times per day and CO<sub>2</sub> level is collected 6 times per day. 

Run the following queries to learn about how to work with and manage nested data in Spark SQL.<br>
In this notebook, you will: 
* Work with hierarchical data
* Use common table expressions (CTE)
* Create new tables based on CTEs
* Use `EXPLODE` to manage nested objects

### Getting started

Run the cell below to set up your classroom environment.

In [0]:
%run ../Includes/Classroom-Setup

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

res1: Boolean = false

res2: Boolean = false

### Create table 

The [Databricks File System (DBFS)](https://docs.databricks.com/data/databricks-file-system.html) is a distributed file system mounted into a Databricks workspace and available on Databricks clusters. In practice, this will allow you to access data that has been mounted to your workspace and interact with that storage using directories and file names instead of storage urls. In this lesson, we'll use data from datasets in object storage that has been mounted to the DBFS. We will create a table and explore some of the optional arguments available to us.

The cell below begins with a `DROP TABLE IF EXISTS`; command. This means that if a table by the given name exists, it will be dropped. If it does not exist, this command does nothing. This will keep our notebook **idempotent**, meaning it could be run more than once without throwing errors or introducing extra files.

In [0]:
%sql
DROP TABLE IF EXISTS DCDataRaw;
CREATE TABLE DCDataRaw
USING parquet                           
OPTIONS (
    PATH "/mnt/training/iot-devices/data-centers/2019-q2-q3"
    )

### View metadata and "Detailed Table Information"

In a previous lesson, we used the `DESCRIBE` command to view metadata. Run the command below to see the output when we attach the optional keyword `EXTENDED`. 

You can find the same information about the schema at the top. Notice that one of our columns contains a `MapType` column, and, within that, a `StructType` field. When working with structured data, like parquet files, and semi-structured data, like JSON files, you will frequently encounter complex data types, like `MapType`, `StructType`, and `ArrayType`. 

In this example, the `MapType` column holds a JSON object that has a `string` as its **key** and a `struct` field as the **value**. As you work through this notebook, we will unnest and explore that data. Learn more about the data types you will be working with in Spark SQL in the [associated docs](https://spark.apache.org/docs/latest/sql-ref-datatypes.html).

**Detailed Table Information** contains information about the table's database name, original source file type and location, and more.

In [0]:
%sql
DESCRIBE EXTENDED DCDataRaw;

col_name,data_type,comment
dc_id,string,null
date,string,null
source,"map,co2_level:array>>",null
,,
# Detailed Table Information,,
Database,default,
Table,dcdataraw,
Owner,root,
Created Time,Fri Dec 04 17:52:59 UTC 2020,
Last Access,UNKNOWN,


### View a sample
It may you help understand the data if we view a few rows. Instead of simply returning the top rows, we can get a random sampling of rows using the function `RAND()` to return random rows and the `LIMIT` keyword to set the number of rows we want to see.

In [0]:
%sql
SELECT * FROM DCDataRaw
ORDER BY RAND()
LIMIT 3;

dc_id,date,source
dc-102,2019-07-16,"Map(sensor-igauge -> List(Sensor attached to the container ceilings, 101.73.53.46, 14, List(22, 14, 14, 20, 9, 21, 12, 22, 12, 18, 17, 15), List(1289, 1441, 1425, 1406, 1367, 1542)), sensor-ipad -> List(Sensor ipad attached to carbon cylinders, 133.40.143.84, 19, List(28, 23, 24, 28, 28, 24, 20, 26, 29, 28, 24, 26), List(1360, 1550, 1325, 1466, 1499, 1301)), sensor-inest -> List(Sensor attached to the factory ceilings, 261.201.235.195, 22, List(22, 25, 20, 26, 23, 16, 22, 18, 23, 19, 25, 20), List(1177, 1472, 1143, 1327, 1162, 1171)), sensor-istick -> List(Sensor embedded in exhaust pipes in the ceilings, 217.127.55.132, 30, List(12, 16, 10, 19, 22, 19, 28, 18, 25, 11, 19, 12), List(1230, 1334, 1291, 1337, 1239, 1309)))"
dc-101,2019-10-02,"Map(sensor-igauge -> List(Sensor attached to the container ceilings, 272.254.144.73, 18, List(14, 4, 9, 6, 13, 12, 17, 14, 10, 10, 12, 17), List(1154, 1404, 1165, 1334, 1169, 1342)), sensor-ipad -> List(Sensor ipad attached to carbon cylinders, 25.238.227.26, 22, List(22, 12, 12, 19, 10, 22, 22, 10, 12, 20, 11, 9), List(1085, 1127, 1208, 1303, 986, 1058)), sensor-inest -> List(Sensor attached to the factory ceilings, 138.190.197.81, 28, List(11, 16, 11, 8, 9, 10, 11, 12, 16, 11, 14, 12), List(1189, 993, 1166, 1047, 1153, 1132)), sensor-istick -> List(Sensor embedded in exhaust pipes in the ceilings, 79.22.267.233, 32, List(1, 20, 9, 10, 5, 11, 5, 9, 10, 9, 14, 11), List(1089, 1197, 1018, 1018, 1177, 1238)))"
dc-104,2019-07-19,"Map(sensor-igauge -> List(Sensor attached to the container ceilings, 197.106.98.60, 15, List(14, 17, 7, 10, 14, 16, 19, 20, 18, 13, 16, 17), List(1134, 1200, 1355, 1018, 1249, 1331)), sensor-ipad -> List(Sensor ipad attached to carbon cylinders, 83.185.145.136, 18, List(17, 10, 13, 16, 7, 13, 25, 12, 14, 15, 25, 16), List(1000, 947, 1114, 829, 1054, 914)), sensor-inest -> List(Sensor attached to the factory ceilings, 250.56.21.177, 23, List(7, 20, 21, 8, 5, 12, 13, 16, 15, 6, 11, 4), List(1073, 1208, 1252, 1261, 1109, 1239)), sensor-istick -> List(Sensor embedded in exhaust pipes in the ceilings, 183.163.134.262, 28, List(27, 18, 25, 16, 19, 19, 17, 15, 18, 21, 13, 13), List(1128, 878, 1014, 1069, 1022, 1096)))"


### Explode a nested object
We can observe from the output that the `source` column contains a nested object with named `key-value` pairs. We'll use `EXPLODE` to get a closer look at the data in that column. 

**`EXPLODE`** is used with arrays and elements of a map expression. When used with an array, it splits the elements into multiple rows. Used with a map, as in this example, it splits the elements of a map into multiple rows and columns and uses the default names, `key` and `value`, to name the new columns. This data structure is mapped such that each `key`, the name of a certain device, holds an object, `value`, containing information about that device.

In [0]:
%sql
SELECT EXPLODE (source)
FROM DCDataRaw;

key,value
sensor-igauge,"List(Sensor attached to the container ceilings, 43.48.56.53, 15, List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12), List(1196, 1360, 1125, 1206, 1342, 1198))"
sensor-ipad,"List(Sensor ipad attached to carbon cylinders, 171.59.65.289, 21, List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12), List(1002, 988, 1137, 1171, 1094, 1206))"
sensor-inest,"List(Sensor attached to the factory ceilings, 189.125.254.115, 29, List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12), List(1266, 1254, 1169, 1204, 1165, 1342))"
sensor-istick,"List(Sensor embedded in exhaust pipes in the ceilings, 27.84.20.121, 37, List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22), List(1098, 1113, 1161, 981, 967, 939))"
sensor-igauge,"List(Sensor attached to the container ceilings, 286.139.183.97, 18, List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3), List(1494, 1385, 1378, 1335, 1480, 1107))"
sensor-ipad,"List(Sensor ipad attached to carbon cylinders, 55.80.146.19, 22, List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12), List(1312, 1386, 1375, 1359, 1396, 1308))"
sensor-inest,"List(Sensor attached to the factory ceilings, 127.239.249.257, 27, List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23), List(1100, 1144, 887, 1120, 1238, 1061))"
sensor-istick,"List(Sensor embedded in exhaust pipes in the ceilings, 173.158.268.37, 30, List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8), List(1387, 1450, 1613, 1357, 1537, 1318))"
sensor-igauge,"List(Sensor attached to the container ceilings, 81.279.151.105, 16, List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21), List(1187, 1141, 1038, 1172, 1094, 1108))"
sensor-ipad,"List(Sensor ipad attached to carbon cylinders, 40.225.54.260, 19, List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5), List(1135, 1166, 1258, 1068, 1165, 1091))"


### Common Table Expressions

Common Table Expressions (CTE) are supported in Spark SQL. A CTE provides a temporary result set which you can then use in a `SELECT` statement. These are different from temporary views in that they cannot be used beyond the scope of a single query. In this case, we will use the CTE to get a closer look at the nested data without writing a new table or view. CTEs use the `WITH` clause to start defining the expression.

Notice that after we explode the source column, we can access individual properties in the `value` field by using dot notation with the property name.

In [0]:
%sql
WITH ExplodeSource  -- specify the name of the result set we will query
AS                  
(                   -- wrap a SELECT statement in parentheses
  SELECT            -- this is the temporary result set you will query
    dc_id,
    to_date(date) AS date,
    EXPLODE (source)
  FROM
    DCDataRaw
)
SELECT             -- write a select statment to query the result set
  key,
  dc_id,
  date,
  value.description,  
  value.ip,
  value.temps,
  value.co2_level
FROM               -- this query is coming from the CTE we named
  ExplodeSource;  
                  

key,dc_id,date,description,ip,temps,co2_level
sensor-igauge,dc-101,2019-07-01,Sensor attached to the container ceilings,43.48.56.53,"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)","List(1196, 1360, 1125, 1206, 1342, 1198)"
sensor-ipad,dc-101,2019-07-01,Sensor ipad attached to carbon cylinders,171.59.65.289,"List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12)","List(1002, 988, 1137, 1171, 1094, 1206)"
sensor-inest,dc-101,2019-07-01,Sensor attached to the factory ceilings,189.125.254.115,"List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12)","List(1266, 1254, 1169, 1204, 1165, 1342)"
sensor-istick,dc-101,2019-07-01,Sensor embedded in exhaust pipes in the ceilings,27.84.20.121,"List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22)","List(1098, 1113, 1161, 981, 967, 939)"
sensor-igauge,dc-101,2019-07-02,Sensor attached to the container ceilings,286.139.183.97,"List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3)","List(1494, 1385, 1378, 1335, 1480, 1107)"
sensor-ipad,dc-101,2019-07-02,Sensor ipad attached to carbon cylinders,55.80.146.19,"List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12)","List(1312, 1386, 1375, 1359, 1396, 1308)"
sensor-inest,dc-101,2019-07-02,Sensor attached to the factory ceilings,127.239.249.257,"List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23)","List(1100, 1144, 887, 1120, 1238, 1061)"
sensor-istick,dc-101,2019-07-02,Sensor embedded in exhaust pipes in the ceilings,173.158.268.37,"List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8)","List(1387, 1450, 1613, 1357, 1537, 1318)"
sensor-igauge,dc-101,2019-07-03,Sensor attached to the container ceilings,81.279.151.105,"List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21)","List(1187, 1141, 1038, 1172, 1094, 1108)"
sensor-ipad,dc-101,2019-07-03,Sensor ipad attached to carbon cylinders,40.225.54.260,"List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5)","List(1135, 1166, 1258, 1068, 1165, 1091)"


### Create Table as Select (CTAS)

CTEs like those in the cell above are temporary and cannot be queried again. In the next cell, we demonstrate how you create a table using the common table expression syntax. 

In Spark SQL, you can populate a new table with input data from a `SELECT` statement. The following is an example where we create a new table, `DeviceData`, using the CTE syntax we used in the previous cell. In this example, we rename the `key` column to `device_type`.

In [0]:
%sql
DROP TABLE IF EXISTS DeviceData;
CREATE TABLE DeviceData                 
USING parquet
WITH ExplodeSource                       -- The start of the CTE from the last cell
AS
  (
  SELECT 
  dc_id,
  to_date(date) AS date,
  EXPLODE (source)
  FROM DCDataRaw
  )
SELECT 
  dc_id,
  key device_type,                       
  date,
  value.description,
  value.ip,
  value.temps,
  value.co2_level
  
FROM ExplodeSource;



Run a `SELECT` all to view the new table.

In [0]:
%sql
SELECT * FROM DeviceData

dc_id,device_type,date,description,ip,temps,co2_level
dc-101,sensor-igauge,2019-07-01,Sensor attached to the container ceilings,43.48.56.53,"List(16, 13, 19, 11, 9, 23, 18, 13, 18, 17, 12, 12)","List(1196, 1360, 1125, 1206, 1342, 1198)"
dc-101,sensor-ipad,2019-07-01,Sensor ipad attached to carbon cylinders,171.59.65.289,"List(26, 17, 19, 13, 9, 12, 10, 12, 1, 13, 16, 12)","List(1002, 988, 1137, 1171, 1094, 1206)"
dc-101,sensor-inest,2019-07-01,Sensor attached to the factory ceilings,189.125.254.115,"List(11, 13, 19, 8, 14, 16, 13, 14, 14, 9, 7, 12)","List(1266, 1254, 1169, 1204, 1165, 1342)"
dc-101,sensor-istick,2019-07-01,Sensor embedded in exhaust pipes in the ceilings,27.84.20.121,"List(20, 18, 20, 18, 11, 14, 17, 24, 17, 15, 19, 22)","List(1098, 1113, 1161, 981, 967, 939)"
dc-101,sensor-igauge,2019-07-02,Sensor attached to the container ceilings,286.139.183.97,"List(3, 1, 7, 8, 2, -4, 7, 1, 0, 8, -2, 3)","List(1494, 1385, 1378, 1335, 1480, 1107)"
dc-101,sensor-ipad,2019-07-02,Sensor ipad attached to carbon cylinders,55.80.146.19,"List(11, 11, 16, 21, 8, 22, 13, 15, 10, 12, 14, 12)","List(1312, 1386, 1375, 1359, 1396, 1308)"
dc-101,sensor-inest,2019-07-02,Sensor attached to the factory ceilings,127.239.249.257,"List(21, 20, 23, 25, 22, 28, 23, 28, 18, 19, 26, 23)","List(1100, 1144, 887, 1120, 1238, 1061)"
dc-101,sensor-istick,2019-07-02,Sensor embedded in exhaust pipes in the ceilings,173.158.268.37,"List(9, 4, 15, 10, 5, 9, 8, 8, 1, 6, 6, 8)","List(1387, 1450, 1613, 1357, 1537, 1318)"
dc-101,sensor-igauge,2019-07-03,Sensor attached to the container ceilings,81.279.151.105,"List(16, 14, 10, 12, 17, 10, 13, 11, 8, 19, 11, 21)","List(1187, 1141, 1038, 1172, 1094, 1108)"
dc-101,sensor-ipad,2019-07-03,Sensor ipad attached to carbon cylinders,40.225.54.260,"List(11, 15, 11, 5, 13, 14, 11, 12, 9, 9, 11, 5)","List(1135, 1166, 1258, 1068, 1165, 1091)"


In [0]:
%run ../Includes/Classroom-Cleanup


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>